<h1>Reproduction Blog: Deep Fruit Detection in Orchards</h1>

Author(s):  
Yulei Qiu (5233178) y.qiu-7@student.tudelft.nl   
Yurui Du (5217849) y.du-7@student.tudelft.nl

<p>The goal of this blog post is to present our attempt to reproduce the paper <a href="https://arxiv.org/abs/1610.03677">Deep Fruit Detection in Orchards</a>, written by Suchet Bargoti and James Underwood, 2016. This paper describes the use of a state-of-the-art object detection framework, Faster R-CNN, in the context of fruit detection in orchards, including mangoes, almonds and apples. Throughout this notebook, we explain our efforts to reproduce Fig. 2 of the paper. We referred to some existing implementation of Faster R-CNN. As we could certainly not use and run them directly, we implemented some new code variants throughout this project.</p>

<p><em>We are one of the reproduction groups of the course CS4240 Deep Learning of Delft University of Technology. More reproduce works of our fellow students can be found <a href="https://reproducedpapers.org/">here</a></em>.</p>

<h2>Architecture</h2>

<p>According to the paper <a href="https://arxiv.org/abs/1506.01497">Faster R-CNN: Towards Real-Time Object Detection with Region Proposal Networks</a>, the architecture of the Faster R-CNN consists of the following parts:</p>
<ol>
<li>Input: images, represented by Height × Width × Depth tensors</li>
<li>Convolutional layer: a pre-trained CNN, ending up with a convolutional feature map</li>
<li>Region Proposal Networks (RPN): takes the feature as input and outputs a set of rectangular object proposals (bounding boxes), which may contain objects</li>
<li>Region of Interest (RoI) Pooling: extracts those features which would correspond to the relevant objects into a new tensor
<li>R-CNN module: classifies the content in the bounding box and adjusts the bounding box coordinates
</ol>

![](https://drive.google.com/uc?id=1wTRGqd3FMwKm8nTjpLhq04CUXACCkYWh)
Source: https://tryolabs.com/blog/2018/01/18/faster-r-cnn-down-the-rabbit-hole-of-modern-object-detection/

<h2>Data</h2>

<p>The dataset is provide by the authors of the paper, which is accessible from <a href="http://data.acfr.usyd.edu.au/ag/treecrops/2016-multifruit/">here</a>. You can download the dataset directly. The labeller is just for visualization.</p>

<p>We choose to work on the almond dataset, which contains 620 images (300 x 300 pixels). The annotations are already done and saved as .csv files in another folder. More specificly, they are bounding boxes including every fruit object in each images. To further extend the dataset, we perform data augmentation, which will be explained in datail later.</p>

<h2>Approach</h2>

<p>There is no existing code for the paper <a href="https://arxiv.org/abs/1610.03677">Deep Fruit Detection in Orchards</a>. But, there is some existing implementation of Faster R-CNN. Therefore, we decide not to do the reproduction from scratch. We refer to the code of Faster R-CNN and implement the network on our own.</p>
<p>We then train the Faster R-CNN and plot the accuracy and loss. The test is done on a smaller dataset. We show the intermediate images in test process and reproduce Fig. 2</p>

<h2>Reproducing Fig. 2</h2>

<p>The figure below shows the Faster R-CNN network in the paper <a href="https://arxiv.org/abs/1610.03677">Deep Fruit Detection in Orchards</a>. We will follow the flow chart and re-implement the network.</p>

![](https://drive.google.com/uc?id=1hIinMOaoM39GJvkk6j9wenYEmGJ7CwGw)
<p>Fig. 2. The Faster R-CNN Network. A 3-channel input image is propagated through a set of convolutional layers, from which Region of Interest boxes are proposed (dashed red boxes, with one high object probability box highlighted as an example). Each box is propagated through fully connected layers, which return their class probability and regresses a finer bounding box around individual objects (solid red boxes). Ground truth from the input image (in blue) is used in the RPN and the R-CNN layers during training. During testing, a class specific detection threshold is applied to the output, followed by Non-Maximum Suppression to remove overlapping results.</p>
Credit: <a href="https://arxiv.org/abs/1506.01497">Faster R-CNN: Towards Real-Time Object Detection with Region Proposal Networks</a

<h3>Convolutional Layers</h3>

<p>The authors experiment with ZF and VGG16 network as the convolutional layers. Here, we use VGG16, which contains 13 convolutional layers. We define VGG16 as the base network here so that the weights of convolutional layers can be shared between the RPN and the R-CNN components.</p>

In [ ]:
def nn_base(input_tensor=None, trainable=False):


    input_shape = (None, None, 3)

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor

    bn_axis = 3

    # Block 1
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    # x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    return x

<h3>Region Proposal Network (RPN)</h3>

<p>Local regions in the feature map are forward propagated into two sibling fully connected layers, a box-regression layer and a box-classification layer. This is the RPN layer, and the fixed number of class agnostic detections are the object proposals.</p>

In [ ]:
def rpn_layer(base_layers, num_anchors):
    """Create a rpn layer
        Step1: Pass through the feature map from base layer to a 3x3 512 channels convolutional layer
                Keep the padding 'same' to preserve the feature map's size
        Step2: Pass the step1 to two (1,1) convolutional layer to replace the fully connected layer
                classification layer: num_anchors (9 in here) channels for 0, 1 sigmoid activation output
                regression layer: num_anchors*4 (36 in here) channels for computing the regression of bboxes with linear activation
    Args:
        base_layers: vgg in here
        num_anchors: 9 in here

    Returns:
        [x_class, x_regr, base_layers]
        x_class: classification for whether it's an object
        x_regr: bboxes regression
        base_layers: vgg in here
    """
    
    x = Conv2D(512, (3, 3), padding='same', activation='relu', kernel_initializer='normal', name='rpn_conv1')(base_layers)

    x_class = Conv2D(num_anchors, (1, 1), activation='sigmoid', kernel_initializer='uniform', name='rpn_out_class')(x)
    x_regr = Conv2D(num_anchors * 4, (1, 1), activation='linear', kernel_initializer='zero', name='rpn_out_regress')(x)

    return [x_class, x_regr, base_layers]

<h3>Region of Interest (RoI) Pooling</h3>

<p>Next, we implement the ROI Pooling. The purpose of ROI Pooli is to perform max pooling on inputs of nonuniform sizes to obtain fixed-size feature maps</p>

In [ ]:
class RoiPoolingConv(Layer):
    '''ROI pooling layer for 2D inputs.
    See Spatial Pyramid Pooling in Deep Convolutional Networks for Visual Recognition,
    K. He, X. Zhang, S. Ren, J. Sun
    # Arguments
        pool_size: int
            Size of pooling region to use. pool_size = 7 will result in a 7x7 region.
        num_rois: number of regions of interest to be used
    # Input shape
        list of two 4D tensors [X_img,X_roi] with shape:
        X_img:
        `(1, rows, cols, channels)`
        X_roi:
        `(1,num_rois,4)` list of rois, with ordering (x,y,w,h)
    # Output shape
        3D tensor with shape:
        `(1, num_rois, channels, pool_size, pool_size)`
    '''
    
    def __init__(self, pool_size, num_rois, **kwargs):

        self.dim_ordering = K.image_data_format()
        self.pool_size = pool_size
        self.num_rois = num_rois

        super(RoiPoolingConv, self).__init__(**kwargs)

    def build(self, input_shape):
        self.nb_channels = input_shape[0][3]   

    def compute_output_shape(self, input_shape):
        return None, self.num_rois, self.pool_size, self.pool_size, self.nb_channels

    def call(self, x, mask=None):

        assert(len(x) == 2)

        # x[0] is image with shape (rows, cols, channels)
        img = x[0]

        # x[1] is roi with shape (num_rois,4) with ordering (x,y,w,h)
        rois = x[1]

        input_shape = K.shape(img)

        outputs = []

        for roi_idx in range(self.num_rois):

            x = rois[0, roi_idx, 0]
            y = rois[0, roi_idx, 1]
            w = rois[0, roi_idx, 2]
            h = rois[0, roi_idx, 3]

            x = K.cast(x, 'int32')
            y = K.cast(y, 'int32')
            w = K.cast(w, 'int32')
            h = K.cast(h, 'int32')

            # Resized roi of the image to pooling size (7x7)
            rs = tf.image.resize(img[:, y:y+h, x:x+w, :], (self.pool_size, self.pool_size))
            outputs.append(rs)
                

        final_output = K.concatenate(outputs, axis=0)

        # Reshape to (1, num_rois, pool_size, pool_size, nb_channels)
        # Might be (1, 4, 7, 7, 3)
        final_output = K.reshape(final_output, (1, self.num_rois, self.pool_size, self.pool_size, self.nb_channels))

        # permute_dimensions is similar to transpose
        final_output = K.permute_dimensions(final_output, (0, 1, 2, 3, 4))

        return final_output
    
    
    def get_config(self):
        config = {'pool_size': self.pool_size,
                  'num_rois': self.num_rois}
        base_config = super(RoiPoolingConv, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

<p>The following block utilizes ROI Pooling to extract fixed-sized feature maps for each proposal, together with final step in Faster R-CNN, where we use these features for classification. A fully-connected layer is used to output a score for each possible object class. The authors of Faster R-CNN takes the feature map for each proposal, flattens it and uses two fully-connected layers of size 4096 with ReLU activation. </p>

In [ ]:
def classifier_layer(base_layers, input_rois, num_rois, nb_classes = 4):
    """Create a classifier layer
    
    Args:
        base_layers: vgg
        input_rois: `(1,num_rois,4)` list of rois, with ordering (x,y,w,h)
        num_rois: number of rois to be processed in one time (4 in here)

    Returns:
        list(out_class, out_regr)
        out_class: classifier layer output
        out_regr: regression layer output
    """    

    input_shape = (num_rois,7,7,512)

    pooling_regions = 7

    # out_roi_pool.shape = (1, num_rois, channels, pool_size, pool_size)
    # num_rois (4) 7x7 roi pooling
    out_roi_pool = RoiPoolingConv(pooling_regions, num_rois)([base_layers, input_rois])

    # Flatten the convlutional layer and connected to 2 FC and 2 dropout
    out = TimeDistributed(Flatten(name='flatten'))(out_roi_pool)
    out = TimeDistributed(Dense(4096, activation='relu', name='fc1'))(out)
    out = TimeDistributed(Dropout(0.5))(out)
    out = TimeDistributed(Dense(4096, activation='relu', name='fc2'))(out)
    out = TimeDistributed(Dropout(0.5))(out)

    # There are two output layer
    # out_class: softmax acivation function for classify the class name of the object
    # out_regr: linear activation function for bboxes coordinates regression
    out_class = TimeDistributed(Dense(nb_classes, activation='softmax', kernel_initializer='zero'), name='dense_class_{}'.format(nb_classes))(out)
    # note: no regression target for bg class
    out_regr = TimeDistributed(Dense(4 * (nb_classes-1), activation='linear', kernel_initializer='zero'), name='dense_regress_{}'.format(nb_classes))(out)

    return [out_class, out_regr]

<h3>Loss function</h3>

<p>Since the RPN have two predictions, classification and regression, we should define the loss functions for them. For regression, the authors use Smooth L1 loss function to increase the convergence rate. For classification, they calculate the loss using binary cross entropy.</p>

<h3>Non-Maximum-Suppression</h3>

This following block implements the Non-Maximum-Suppression (NMS) to handle overlapping detections. The original code is by <a href="https://github.com/rbgirshick/voc-dpm/blob/master/test/nms.m">Felzenszwalb et al </a>. Here we use the faster version which replaces for loops with vectorized operations for 100 times speed-up. 

The overlapping threshold is adopting the optimized value for fruit detection suggested by <a href="https://arxiv.org/abs/1610.03677">Deep Fruit Detection in Orchards</a>.

In [ ]:
def non_max_suppression_fast(boxes, probs, overlap_thresh=0.3, max_boxes=300):
    # code used from here: http://www.pyimagesearch.com/2015/02/16/faster-non-maximum-suppression-python/
    # if there are no boxes, return an empty list

     
    if len(boxes) == 0:
        return []

    # grab the coordinates of the bounding boxes
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]

    np.testing.assert_array_less(x1, x2)
    np.testing.assert_array_less(y1, y2)

    # if the bounding boxes integers, convert them to floats --
    # this is important since we'll be doing a bunch of divisions
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")

    # initialize the list of picked indexes	
    pick = []

    # calculate the areas
    area = (x2 - x1) * (y2 - y1)

    # sort the bounding boxes 
    idxs = np.argsort(probs)

    # keep looping while some indexes still remain in the indexes
    # list
    while len(idxs) > 0:
        # grab the last index in the indexes list and add the
        # index value to the list of picked indexes
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)

        # find the intersection

        xx1_int = np.maximum(x1[i], x1[idxs[:last]])
        yy1_int = np.maximum(y1[i], y1[idxs[:last]])
        xx2_int = np.minimum(x2[i], x2[idxs[:last]])
        yy2_int = np.minimum(y2[i], y2[idxs[:last]])

        ww_int = np.maximum(0, xx2_int - xx1_int)
        hh_int = np.maximum(0, yy2_int - yy1_int)

        area_int = ww_int * hh_int

        # find the union
        area_union = area[i] + area[idxs[:last]] - area_int

        # compute the ratio of overlap
        overlap = area_int/(area_union + 1e-6)

        # delete all indexes from the index list that have
        idxs = np.delete(idxs, np.concatenate(([last],
            np.where(overlap > overlap_thresh)[0])))

        if len(pick) >= max_boxes:
            break

    # return only the bounding boxes that were picked using the integer data type
    boxes = boxes[pick].astype("int")
    probs = probs[pick]
    return boxes, probs


        w1 = np.round(w1)
        h1 = np.round(h1)
        return np.stack([x1, y1, w1, h1])
    except Exception as e:
        print(e)
        return X


# Training

This is our training part of the code. Here we demonstrate the training process by training a model for almond detection. 

In [ ]:
#------Change path to fit one's own needs.
base_path = './'

train_path1 =  './almonds/' # data folder
#------Change path to fit one's own needs.

num_rois = 4 # Number of RoIs to process at once.

# Augmentation flag
horizontal_flips = True # Augment with horizontal flips in training. 
vertical_flips = True   # Augment with vertical flips in training. 
rot_90 = True           # Augment with 90 degree rotations in training. 

#------Change path of weight files, configuration files, and record files
output_weight_path = os.path.join(base_path, 'model/model_frcnn_vgg.hdf5')

record_path = os.path.join(base_path, 'model/record.csv') # Record data (used to save the losses, classification accuracy and mean average precision)

base_weight_path = os.path.join(base_path, 'model/vgg16_weights_tf_dim_ordering_tf_kernels.h5')

config_output_filename = os.path.join(base_path, 'model_vgg_config.pickle')
#------Change path of weight files, configuration files, and record files

Here we apply the same data augmentation as suggested in <a href="https://arxiv.org/abs/1610.03677">Deep Fruit Detection in Orchards</a> by setting up the correct configurations. To be more specific, we perform data augmentation with a 90° rotation on training images with both horizontal and vertical flips.

In [ ]:
C = Config()

C.use_horizontal_flips = horizontal_flips
C.use_vertical_flips = vertical_flips
C.rot_90 = rot_90

C.record_path = record_path
C.model_path = output_weight_path
C.num_rois = num_rois

C.base_net_weights = base_weight_path

Keep track of the time loading images, as sometimes it can be really slow on Colab.

In [ ]:
st = time.time()
train_imgs, classes_count, class_mapping = get_data(train_path1)
print()
print('Spend %0.2f mins to load the data' % ((time.time()-st)/60) )

Examine whether our data was correctly loaded. Save the configuration for testing on the same model. 

In [ ]:
if 'bg' not in classes_count:
	classes_count['bg'] = 0
	class_mapping['bg'] = len(class_mapping)
# e.g.
#    classes_count: {'Car': 2383, 'Mobile phone': 1108, 'Person': 3745, 'bg': 0}
#    class_mapping: {'Person': 0, 'Car': 1, 'Mobile phone': 2, 'bg': 3}
C.class_mapping = class_mapping

print('Training images per class:')
pprint.pprint(classes_count)
print('Num classes (including bg) = {}'.format(len(classes_count)))
print(class_mapping)

# Save the configuration
with open(config_output_filename, 'wb') as config_f:
	pickle.dump(C,config_f)
	print('Config has been written to {}, and can be loaded when testing to ensure correct results'.format(config_output_filename))

Randomly shuffle the training images, always using random seed 42 for reproducible results!

In [ ]:
# Shuffle the images with seed
random.seed(42)
random.shuffle(train_imgs)

print('Num train samples (images) {}'.format(len(train_imgs)))

Get anchors for training images' ground truth on bounding boxes (locations, size). Use 'next' to generate iterables for getting new data to save memory for large training sets.

In [ ]:
data_gen_train = get_anchor_gt(train_imgs, C, get_img_output_length, mode='train')
X, Y, image_data, debug_img, debug_num_pos = next(data_gen_train)

Now we can take a look at our ground truth bounding boxes in the training image!

![](https://drive.google.com/uc?id=1b0WsRNZjfJcrOOOXNSE9eiwLRiBJqlia)

Initialise our network by setting up correct parameters as suggested by <a href="https://arxiv.org/abs/1610.03677">Deep Fruit Detection in Orchards</a>.

In [ ]:
input_shape_img = (None, None, 3)

img_input = Input(shape=input_shape_img)
roi_input = Input(shape=(None, 4))

# define the base network (VGG16 here, can be Resnet50, Inception, etc)
shared_layers = nn_base(img_input, trainable=True)

In [ ]:
num_anchors = len(C.anchor_box_scales) * len(C.anchor_box_ratios) # 9
rpn = rpn_layer(shared_layers, num_anchors)

classifier = classifier_layer(shared_layers, roi_input, C.num_rois, nb_classes=len(classes_count))

model_rpn = Model(img_input, rpn[:2])
model_classifier = Model([img_input, roi_input], classifier)

# this is a model that holds both the RPN and the classifier, used to load/save weights for the models
model_all = Model([img_input, roi_input], rpn[:2] + classifier)

# Because the google colab can only run the session several hours one time (then you need to connect again), 
# we need to save the model and load the model to continue training
if not os.path.isfile(C.model_path):
    #If this is the begin of the training, load the pre-traind base network such as vgg-16
    try:
        print('This is the first time of your training')
        print('loading weights from {}'.format(C.base_net_weights))
        model_rpn.load_weights(C.base_net_weights, by_name=True)
        model_classifier.load_weights(C.base_net_weights, by_name=True)
    except:
        print('Could not load pretrained model weights. Weights can be found in the keras application folder \
            https://github.com/fchollet/keras/tree/master/keras/applications')
    
    # Create the record.csv file to record losses, acc and mAP
    record_df = pd.DataFrame(columns=['mean_overlapping_bboxes', 'class_acc', 'loss_rpn_cls', 'loss_rpn_regr', 'loss_class_cls', 'loss_class_regr', 'curr_loss', 'elapsed_time', 'mAP'])
else:
    # If this is a continued training, load the trained model from before
    print('Continue training based on previous trained model')
    print('Loading weights from {}'.format(C.model_path))
    model_rpn.load_weights(C.model_path, by_name=True)
    model_classifier.load_weights(C.model_path, by_name=True)
    
    # Load the records
    record_df = pd.read_csv(record_path)

    r_mean_overlapping_bboxes = record_df['mean_overlapping_bboxes']
    r_class_acc = record_df['class_acc']
    r_loss_rpn_cls = record_df['loss_rpn_cls']
    r_loss_rpn_regr = record_df['loss_rpn_regr']
    r_loss_class_cls = record_df['loss_class_cls']
    r_loss_class_regr = record_df['loss_class_regr']
    r_curr_loss = record_df['curr_loss']
    r_elapsed_time = record_df['elapsed_time']
    r_mAP = record_df['mAP']

    print('Already train %dK batches'% (len(record_df)))

In [ ]:
optimizer = Adam(lr=1e-5)
optimizer_classifier = Adam(lr=1e-5)
model_rpn.compile(optimizer=optimizer, loss=[rpn_loss_cls(num_anchors), rpn_loss_regr(num_anchors)])
model_classifier.compile(optimizer=optimizer_classifier, loss=[class_loss_cls, class_loss_regr(len(classes_count)-1)], metrics={'dense_class_{}'.format(len(classes_count)): 'accuracy'})
model_all.compile(optimizer='sgd', loss='mae')

Due to the limitations of our group's computing power at hand, we only train this model with 70 epochs with the length of 100 for each epoch. The following figures show the training result. They are mean number of the overleaping bboxes, accuracy, and losses of regression and classification.

![](https://drive.google.com/uc?id=10aQfkjg5tmMcMCTd-CgKGJmpAS4LPvt1)  
![](https://drive.google.com/uc?id=1GhJRSYALbf7aIXUbFJqr9Pk36DG4FW4F)  
![](https://drive.google.com/uc?id=1lP9oxoosQa9qBNWPGabzavCkAheKgzJ8)  
![](https://drive.google.com/uc?id=14wI6gu0lC2dYfyDmmAggZzEurFp6c6An)

As can be seen from these figures, our model is not fully converged because of the limited number of epochs. However, as our primary reproduction goal is to obtain the complete network of our reference paper, we believe this result is sufficient to show that our reconstructed model implements the correct architecture of the network and can be used to train fruit detectors.

### Testing

The steps of testing is simple: 

1. Load the model's configuration and testing sets.
2. For each testing image, give predicted bounding boxes.
3. Compare the predicted bounding boxes with the ground truth label in terms of their locations and sizes.
4. Compute mean average precision (MAP), as is done by the reference paper.

The following code blocks first show one image with predicted bounding boxes before the non-maximum suppression (NMS), then one after the NMS. Along with the input image shown before, we complete the reproduction of Fig 2. in our main reference paper.

In [ ]:
bbox_threshold = 0.1
i = 1
test_img_names_path = os.path.join(test_path1, 'sets/test.txt')
test_img_path = os.path.join(test_path1, 'images')

with open(test_img_names_path,'r') as f:
    print('Parsing testing files')
    lines = [line.rstrip('\n') for line in f]
    for line in lines: # img_name

        # Print process
        sys.stdout.write('\r'+'idx=' + str(i))
        i += 1
        st = time.time()
        filepath = os.path.join(test_img_path, line + '.png')
        img = cv2.imread(filepath)
        X, ratio = format_img(img, C)
    
        X = np.transpose(X, (0, 2, 3, 1))

        # get output layer Y1, Y2 from the RPN and the feature maps F
        # Y1: y_rpn_cls
        # Y2: y_rpn_regr
        [Y1, Y2, F] = model_rpn.predict(X)

        # Get bboxes by applying NMS 
        # R.shape = (300, 4)
        R = rpn_to_roi(Y1, Y2, C, K.image_data_format(), overlap_thresh=0.2)

        # convert from (x1,y1,x2,y2) to (x,y,w,h)
        R[:, 2] -= R[:, 0]
        R[:, 3] -= R[:, 1]

        # apply the spatial pyramid pooling to the proposed regions
        bboxes = {}
        probs = {}
        for jk in range(R.shape[0]//C.num_rois + 1):
          ROIs = np.expand_dims(R[C.num_rois*jk:C.num_rois*(jk+1), :], axis=0)
          if ROIs.shape[1] == 0:
              break

          if jk == R.shape[0]//C.num_rois:
              #pad R
              curr_shape = ROIs.shape
              target_shape = (curr_shape[0],C.num_rois,curr_shape[2])
              ROIs_padded = np.zeros(target_shape).astype(ROIs.dtype)
              ROIs_padded[:, :curr_shape[1], :] = ROIs
              ROIs_padded[0, curr_shape[1]:, :] = ROIs[0, 0, :]
              ROIs = ROIs_padded

          [P_cls, P_regr] = model_classifier_only.predict([F, ROIs])

          # Calculate bboxes coordinates on resized image
          for ii in range(P_cls.shape[1]):
              # Ignore 'bg' class
              if np.max(P_cls[0, ii, :]) < bbox_threshold or np.argmax(P_cls[0, ii, :]) == (P_cls.shape[2] - 1):
                  continue

              cls_name = class_mapping[np.argmax(P_cls[0, ii, :])]

              if cls_name not in bboxes:
                  bboxes[cls_name] = []
                  probs[cls_name] = []

              (x, y, w, h) = ROIs[0, ii, :]

              cls_num = np.argmax(P_cls[0, ii, :])
              try:
                  (tx, ty, tw, th) = P_regr[0, ii, 4*cls_num:4*(cls_num+1)]
                  tx /= C.classifier_regr_std[0]
                  ty /= C.classifier_regr_std[1]
                  tw /= C.classifier_regr_std[2]
                  th /= C.classifier_regr_std[3]
                  x, y, w, h = apply_regr(x, y, w, h, tx, ty, tw, th)
              except:
                  pass
              bboxes[cls_name].append([C.rpn_stride*x, C.rpn_stride*y, C.rpn_stride*(x+w), C.rpn_stride*(y+h)])
              probs[cls_name].append(np.max(P_cls[0, ii, :]))

        all_dets = []

        for key in bboxes:
            bbox = np.array(bboxes[key])

            for ijk in range(bbox.shape[0]):
                (x1, y1, x2, y2) = bbox[ijk,:]
                #det = {'x1': x1, 'x2': x2, 'y1': y1, 'y2': y2, 'class': key, 'prob': new_probs[jk]}
                #all_dets.append(det)

                # Calculate real coordinates on original image
                (real_x1, real_y1, real_x2, real_y2) = get_real_coordinates(ratio, x1, y1, x2, y2)
                
                cv2.rectangle(img,(real_x1, real_y1), (real_x2, real_y2), (int(class_to_color[key][0]), int(class_to_color[key][1]), int(class_to_color[key][2])),4)
                
        print('Elapsed time = {}'.format(time.time() - st))
        plt.figure(figsize=(10,10))
        plt.grid()
        plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
        plt.show()


This is the pre-NMS prediction. Many bounding boxes are overlapping with others, which is exactly why we need to perform NMS to reduce the overlapping boxes and save one box for one fruit.  
![](https://drive.google.com/uc?id=1pgfEyEWRv-472AgcncqxOu-qDGp9cDbc)

In [ ]:
        for key in bboxes:
            bbox = np.array(bboxes[key])

            new_boxes, new_probs = non_max_suppression_fast(bbox, np.array(probs[key]), overlap_thresh=0.3)
            #print(new_boxes.shape[0])
            for jk in range(new_boxes.shape[0]):
                (x1, y1, x2, y2) = new_boxes[jk,:]
                det = {'x1': x1, 'x2': x2, 'y1': y1, 'y2': y2, 'class': key, 'prob': new_probs[jk]}
                all_dets.append(det)

                # Calculate real coordinates on original image
                (real_x1, real_y1, real_x2, real_y2) = get_real_coordinates(ratio, x1, y1, x2, y2)
                
                cv2.rectangle(img,(real_x1, real_y1), (real_x2, real_y2), (int(class_to_color[key][0]), int(class_to_color[key][1]), int(class_to_color[key][2])),4)

                textLabel = '{}: {}'.format(key,int(100*new_probs[jk]))
                all_dets.append((key,100*new_probs[jk]))

                (retval,baseLine) = cv2.getTextSize(textLabel,cv2.FONT_HERSHEY_COMPLEX,1,1)
                textOrg = (real_x1, real_y1-0)
                
                #cv2.rectangle(img, (textOrg[0] - 5, textOrg[1]+baseLine - 5), (textOrg[0]+retval[0] + 5, textOrg[1]-retval[1] - 5), (0, 0, 0), 1)
                #cv2.rectangle(img, (textOrg[0] - 5,textOrg[1]+baseLine - 5), (textOrg[0]+retval[0] + 5, textOrg[1]-retval[1] - 5), (255, 255, 255), -1)
                #cv2.putText(img, textLabel, textOrg, cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 0), 1)
                

Much better now, isn't it? This would be the end of our reproduction of fig 2. in the main reference paper as we finished building the complete network architecture and showing all intermediate and final results.  
![](https://drive.google.com/uc?id=1OceP_Yf7n3V7Ol53AbMcEBFvbE4BYdVm)

Determine whether the prediected bounding boxes match with the ground truth label.

In [ ]:
def get_map(pred, gt, f):
	T = {}
	P = {}
	fx, fy = f

	for bbox in gt:
		bbox['bbox_matched'] = False

	pred_probs = np.array([s['prob'] for s in pred])
	box_idx_sorted_by_prob = np.argsort(pred_probs)[::-1]

	for box_idx in box_idx_sorted_by_prob:
		pred_box = pred[box_idx]
		pred_class = pred_box['class']
		pred_x1 = pred_box['x1']
		pred_x2 = pred_box['x2']
		pred_y1 = pred_box['y1']
		pred_y2 = pred_box['y2']
		pred_prob = pred_box['prob']
		if pred_class not in P:
			P[pred_class] = []
			T[pred_class] = []
		P[pred_class].append(pred_prob)
		found_match = False

		for gt_box in gt:
			gt_class = gt_box['class']
			gt_x1 = gt_box['x1']/fx
			gt_x2 = gt_box['x2']/fx
			gt_y1 = gt_box['y1']/fy
			gt_y2 = gt_box['y2']/fy
			gt_seen = gt_box['bbox_matched']
			if gt_class != pred_class:
				continue
			if gt_seen:
				continue
			iou_map = iou((pred_x1, pred_y1, pred_x2, pred_y2), (gt_x1, gt_y1, gt_x2, gt_y2))
			if iou_map >= 0.2: # according to deep fruit paper, this is enough
				found_match = True
				gt_box['bbox_matched'] = True
				break
			else:
				continue

		T[pred_class].append(int(found_match))

	for gt_box in gt:
		if not gt_box['bbox_matched']:# and not gt_box['difficult']:
			if gt_box['class'] not in P:
				P[gt_box['class']] = []
				T[gt_box['class']] = []

			T[gt_box['class']].append(1)
			P[gt_box['class']].append(0)

	return T, P

Now we can compute the MAP score according to the calculated bounding boxes for images and their corresponding ground truth.

In [ ]:
T = {}
P = {}
mAPs = []
for idx, img_data in enumerate(test_imgs):
    print('{}/{}'.format(idx,len(test_imgs)))
    st = time.time()
    filepath = img_data['filepath']

    img = cv2.imread(filepath)

    X, fx, fy = format_img_map(img, C)

    # Change X (img) shape from (1, channel, height, width) to (1, height, width, channel)
    X = np.transpose(X, (0, 2, 3, 1))

    # get the feature maps and output from the RPN
    [Y1, Y2, F] = model_rpn.predict(X)


    R = rpn_to_roi(Y1, Y2, C, K.image_data_format(), overlap_thresh=0.7)

    # convert from (x1,y1,x2,y2) to (x,y,w,h)
    R[:, 2] -= R[:, 0]
    R[:, 3] -= R[:, 1]

    # apply the spatial pyramid pooling to the proposed regions
    bboxes = {}
    probs = {}

    for jk in range(R.shape[0] // C.num_rois + 1):
        ROIs = np.expand_dims(R[C.num_rois * jk:C.num_rois * (jk + 1), :], axis=0)
        if ROIs.shape[1] == 0:
            break

        if jk == R.shape[0] // C.num_rois:
            # pad R
            curr_shape = ROIs.shape
            target_shape = (curr_shape[0], C.num_rois, curr_shape[2])
            ROIs_padded = np.zeros(target_shape).astype(ROIs.dtype)
            ROIs_padded[:, :curr_shape[1], :] = ROIs
            ROIs_padded[0, curr_shape[1]:, :] = ROIs[0, 0, :]
            ROIs = ROIs_padded

        [P_cls, P_regr] = model_classifier_only.predict([F, ROIs])

        # Calculate all classes' bboxes coordinates on resized image (300, 400)
        # Drop 'bg' classes bboxes
        for ii in range(P_cls.shape[1]):

            # If class name is 'bg', continue
            if np.argmax(P_cls[0, ii, :]) == (P_cls.shape[2] - 1):
                continue

            # Get class name
            cls_name = class_mapping[np.argmax(P_cls[0, ii, :])]

            if cls_name not in bboxes:
                bboxes[cls_name] = []
                probs[cls_name] = []

            (x, y, w, h) = ROIs[0, ii, :]

            cls_num = np.argmax(P_cls[0, ii, :])
            try:
                (tx, ty, tw, th) = P_regr[0, ii, 4 * cls_num:4 * (cls_num + 1)]
                tx /= C.classifier_regr_std[0]
                ty /= C.classifier_regr_std[1]
                tw /= C.classifier_regr_std[2]
                th /= C.classifier_regr_std[3]
                x, y, w, h = roi_helpers.apply_regr(x, y, w, h, tx, ty, tw, th)
            except:
                pass
            bboxes[cls_name].append([16 * x, 16 * y, 16 * (x + w), 16 * (y + h)])
            probs[cls_name].append(np.max(P_cls[0, ii, :]))

    all_dets = []

    for key in bboxes:
        bbox = np.array(bboxes[key])

        # Apply non-max-suppression on final bboxes to get the output bounding boxe
        new_boxes, new_probs = non_max_suppression_fast(bbox, np.array(probs[key]), overlap_thresh=0.3)
        for jk in range(new_boxes.shape[0]):
            (x1, y1, x2, y2) = new_boxes[jk, :]
            det = {'x1': x1, 'x2': x2, 'y1': y1, 'y2': y2, 'class': key, 'prob': new_probs[jk]}
            all_dets.append(det)


    print('Elapsed time = {}'.format(time.time() - st))
    t, p = get_map(all_dets, img_data['bboxes'], (fx, fy))
    for key in t.keys():
        if key not in T:
            T[key] = []
            P[key] = []
        T[key].extend(t[key])
        P[key].extend(p[key])
    all_aps = []
    for key in T.keys():
        ap = average_precision_score(T[key], P[key])
        print('{} AP: {}'.format(key, ap))
        all_aps.append(ap)
    print('mAP = {}'.format(np.mean(np.array(all_aps))))
    mAPs.append(np.mean(np.array(all_aps)))

print()
print('mean average precision:', np.mean(np.array(mAPs)))


For this test, our MAP score is 0.66, which is slightly less than 0.72 from our reference paper. Considering we only train our model with 70 epochs and our model is not fully converged yet, we believe our results basically match with the paper's results, proving our implementation of the network architecture is correct.

Above is our reproduction of Fig. 2 of <a href="https://arxiv.org/abs/1610.03677">Deep Fruit Detection in Orchards</a>, which fits the criterion of "new code variant". 

We borrowed a page from this <a href="Faster RCNN</a>https://github.com/rbgirshick/py-faster-rcnn">Faster RCNN</a>. However, our implementation is different in:

1. The faster RCNN code is not straightforward runable on our computers because it contains lots of out-dated functions and also needs strict configurations in terms software versions and hardware requirements. This makes the original code very difficult for beginners to grasp the essence of it as setting up the correct environment to merely run its demo is already taxing work. We adapt this code so we can directly run it on Google Colab, making our code readable even for beginners.

2. We reimplement the network architecture with TensorFlow. We make necessary chages so the model's parameters are the same as which suggested by  <a href="https://arxiv.org/abs/1610.03677">Deep Fruit Detection in Orchards</a>. We rewrite the data loader, training and testing parts to fit our own training/testing purposes for fruit detection.  

We then perform ablation study from two aspects: transfer learning and data augmentation.

### Ablation Study 1: Transfer Learning

For transfer learning, we compare the pre-trained model from almond with a model directly initialised from ImageNet. The utility of transfer learning can be examined by the detection performance of both models on the set of apples. Instead of comparing the detection performance v.s the number of training images as the reference paper does, we want to find out whether transfer learning is still effective when the pre-trained model is not fully converged due to a lack of computing resources. 

Our result shows that the MAP score for the transfer learning model is 0.79 and 0.71 for model initialised from ImageNet. The MAP Score is computed from the average of 3 trials each has 70 epochs with a length of 100 steps. Hence, we believe transfer learning may still be effective even when the pre-trained model is not fully converged.

|  Trial  | MAP(Pre-trained) | MAP(ImageNet) |
|:-------:|:----------------:|:-------------:|
|    1    |       0.81       |      0.72     |
|    2    |       0.78       |      0.71     |
|    3    |       0.79       |      0.72     |
| Average |       0.79       |      0.72     |

### Ablation Study 2: Data Augmentation

Data augmentation is a common way to expand the variability of the training data by artificially enlarging the dataset using label-preserving transformations. The process increases the networks capability to generalise and reduces overfitting. Here we apply horizontal flips, vertical flips and rotation of 90 degree on the training set. In the origin paper, the authors applied data augmentation on apple and mango detection. We want to explore the effect of data augmentation when it comes to almond detection.

Our result shows that the MAP score for the model with and without data augmentation is 0.70 and 0.71, respectively. For apple and mango detection, the authors found that in most cases, with data augmentation, the network reached a fixed detection performance with less than half the number of training samples. Data augmentation does not improve the score significantly for almond detection. The reason of this may be the fact that almonds have small size, making it hard for the network to classify them. Hence, the data augmentation cannot improve the performance for almond detection.

### Conclusion

During the project, we explore the architecture of Faster R-CNN and create a Faster R-CNN model for fruit detection. As reported in previous sections，our model's MAP score detecting almonds is 0.66, which is slightly less than 0.72 in our reference paper due to limited number of training epochs. This score confirms the conslusion in the origin paper that Faster R-CNN works well on fruit detection, also proves our implementation is correct as our intermiediate and final results resembles what are shown in Fig 2 in the reference paper. 

After reproducing Fig 2, We perform the ablation study on transfer learning and data augmentation. The former one indicates that our model pre-trained using almond dataset works better for detecting apples compared to a model with initial weights from ImageNet when the number of epochs of transfer learning limited. This makes sense as the pre-trained model provides a better prior knowledge for fruit detection, which also suggests the detection of apples and almonds may be similar in some way. The ablation study on the effects of data augmentation shows that data augmentation for almond detection cannot significantly improve the detection performance of apples.